In [1]:
from sklearn.ensemble import RandomForestClassifier
import pickle
import pandas as pd
import numpy as np

with open('/mnt/c/Users/resha/Documents/Github/balancing_framework/spy5m_bintp_labelled.pkl', 'rb') as f:
    df_original = pickle.load(f)
with open('/mnt/c/Users/resha/Documents/Github/balancing_framework/spy5m_bintp004_episodes_fracdiff.pkl', 'rb') as f:
    df_fd = pickle.load(f)
with open('/mnt/c/Users/resha/Documents/Github/balancing_framework/spy5m_labelled_episodes_ta.pkl', 'rb') as f:
    df_ta = pickle.load(f)
    df_ta['label'] = df_original['tp_0.004'][df_ta.index]
with open('/mnt/c/Users/resha/Documents/Github/balancing_framework/spy5m_ta_fracdiff.pkl', 'rb') as f:
    df_fd_ta = pickle.load(f)
    df_fd_ta['label'] = df_original['tp_0.004'][df_ta.index]
# PZ algorithm has some look ahead so remove the episode labels, will be uesd only for some kind of analysis afterwards
# df = df_original.drop(columns=['episode']) 
df = df_original[["volume", "vwap", "open", "close", "high", "low", "transactions", "tp_0.004"]].rename(columns={"tp_0.004": "label"}) # 0.01 0.001
# df

X = pd.DataFrame()
y = df['label']
data_forms = ['ta_original', 'ta_frac_diff']

del df_original
del df_fd

for data_form in data_forms:
    if data_form == 'frac_diff':
        X = X.join(df_fd.drop(columns=['label']).add_suffix(f'_{data_form}'), how='outer')
    elif data_form == 'original':
        X = X.join(df.drop(columns=['label']).add_suffix(f'_{data_form}'), how='outer')
    elif data_form == 'first_order_diff':
        # if wanted to omit cols from diff
        # diff = df.drop(['volume', 'transactions', 'label'], axis=1).diff()
        # diff = diff.join(df[['volume', 'transactions']])
        diff = df.drop(['label'], axis=1).diff().add_suffix(f'_{data_form}')
        X = X.join(diff, how='outer')
    elif data_form == 'ta_original':
        X = X.join(df_ta.drop(columns=['label']).add_suffix(f'_{data_form}'), how='outer')
        del df_ta
    elif data_form == 'ta_fod':
        diff = df_ta.drop(['label'], axis=1).diff().add_suffix(f'_{data_form}')
        X = X.join(diff, how='outer')
    elif data_form == 'ta_frac_diff':
        X = X.join(df_fd_ta.drop(columns=['label']).add_suffix(f'_{data_form}'), how='outer')
        del df_fd_ta
        
        
X.dropna(inplace=True)
# drop y with index not in X
y = y[y.index.isin(X.index)]


In [ ]:
clf = RandomForestClassifier(n_estimators=50, max_depth=32)
clf.fit(X, y)